# <center>Курсовой проект<br>«Введение в обработку естественного языка»</center>

**Задание**: обучить чат-бота в Telegram.

Чат-бот будет иметь следующие функции:
 - 1) определение, является ли введенное сообщение отзывом на приложение и в случае, если является, определив тональность отзыва,  выдавать соответствующий ответ от команды разработчиков;
 - 2) включение по команде музыки с youtube;
 - 3) Small-Talk.
 
1): предстоит обучить два классификатора:
 - Message Analyzer будет определять, является ли сообщение отзывом на приложение («Отзыв» или не «Отзыв»), 
 - Sentiment Analyzer будет оценивать тональность отзыва (отзыв отрицательный, нейтральный или положительный ).

2) и 3): реализация будет произведена с помощью Dialogflow.

<img src="img/scheme.png"/>

In [1]:
import re
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

import warnings 
warnings.filterwarnings('ignore', category=Warning)

# 1. Разработка классификаторов Message Analyzer и Sentiment Analyzer

## 1.1. Сбор данных для обучения классификаторов

В качестве отрицательных примеров для класса «Отзыв» соберем сообщения из переписок в мессенджере QIP. Они будут вполне релевантны для Small-Talk.

Каждая переписка представляет собой текстовый файл достаточно простой структуры. Напишем функцию, которая считывает этот файл, убирает из него линии-разделители, оставляет только сообщения и заносит их в датафрейм – аккаунты и дата-время сообщения не требуются:

In [2]:
LINE_RE = r'--------------------------------------[>|<]-'
DATE_RE = r'\([0-9]*:[0-9]*:[0-9]*\s[0-9]*\/[0-9]*\/[0-9]*\)'

def log2df(log_file):

    log_text = ''
    with open(log_file, 'r') as f:
        for line in f:
            log_text += line
    
    log_list = re.split(LINE_RE, log_text)
    
    log_df = pd.DataFrame({'Content': log_list})
    log_df = log_df[log_df['Content'] != '']

    log_df[['User', 'Message']] = log_df['Content'].str.split(DATE_RE, n=1, expand=True)

    log_df.drop(columns=['Content', 'User'], inplace=True)
    log_df['Message'] = log_df['Message'].str.strip()
    
    return log_df

Обрабатываем этой функцией все имеющиеся файлы переписок и объединяем результаты в единый датафрейм:

In [3]:
logs = pd.DataFrame(columns=['Message'])
for log_file in tqdm(glob('data/_qip/*.txt')):
    logs = pd.concat([logs, log2df(log_file)], ignore_index=True)  
    
logs = logs.sample(frac=1., random_state=21).reset_index(drop=True)

100%|████████████████████████████████████████| 202/202 [00:09<00:00, 20.45it/s]


Собранные сообщения:

In [4]:
logs.head(n=5)

,Message
0,"ок, только я в Августе свалю..."
1,*SCRATCH*я тожы
2,?
3,жаль
4,ок


Их количество:

In [5]:
logs.shape[0]

19804

Позитивные примеры для класса «Отзыв» возьмем из непосредственно файла с отзывами на приложение:

In [6]:
reviews = pd.read_excel('data/отзывы за лето.xls', names=['Rating', 'Message', 'Date'])
reviews.head(n=5)

,Rating,Message,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


Рейтинги приложения:

In [7]:
reviews['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Исключим отзывы с рейтингом в 3 балла. Как положительные отзывы возьмем отзывы с рейтингом в 4 и 5 баллов, отрицательные – с рейтингом в 1 и 2 балла:

In [8]:
reviews = reviews[reviews['Rating'] != 3]
reviews['target'] = reviews['Rating'] > 3
reviews['target'] = reviews['target'].astype(np.uint8)

Количество отзывов – сопоставимо с количеством сообщений из переписок:

In [9]:
reviews.shape[0]

19748

Объединим сообщения и отзывы в один датафрейм с меткой, что из них является отзывом:

In [10]:
logs_and_reviews = logs.copy()
logs_and_reviews.columns=['Message']
logs_and_reviews['is_review'] = 0
logs_and_reviews.head()

,Message,is_review
0,"ок, только я в Августе свалю...",0
1,*SCRATCH*я тожы,0
2,?,0
3,жаль,0
4,ок,0


In [11]:
logs_and_reviews = pd.concat([logs_and_reviews, reviews[['Message']]],
                             sort=False, ignore_index=True)
logs_and_reviews['is_review'] = logs_and_reviews['is_review'].fillna(1).astype(np.uint8)
logs_and_reviews.head(5)

,Message,is_review
0,"ок, только я в Августе свалю...",0
1,*SCRATCH*я тожы,0
2,?,0
3,жаль,0
4,ок,0


## 1.2. Обучение классификаторов

In [12]:
import nltk
from pymorphy2 import MorphAnalyzer

Зададим базовое множество стоп-слов – из предлогов, соединительных союзов и частиц:

In [13]:
morpher = MorphAnalyzer()

basic_stop = {'без', 'безо', 'близ', 'в', 'во', 'вместо', 'вне',
        'для', 'до', 'за', 'из', 'изо', 'из-за', 'из-под', 
        'к', 'ко', 'кроме', 'между', 'меж', 'на', 'над', 'надо',
        'о', 'об', 'обо', 'от', 'ото', 'перед', 'передо', 'пред', 'предо',
        'по', 'под', 'подо', 'при', 'про', 'ради', 'с', 'со',
        'сквозь', 'среди', 'у', 'через', 'чрез', 'и', 'или', 'же'}

Функции для нормализации слов и предобработки текста:

In [14]:
def normalize_word(word):
    return morpher.parse(word)[0].normal_form

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[*.,\n\t]', ' ', text)
    text_list = nltk.word_tokenize(str(text))
    text_list = [normalize_word(word) \
                 for word in text_list if word not in basic_stop]
    text = ' '.join(text_list)
#    text = text.replace(' не ', ' не не')   
    return text

Напишем класс для объектов Message Analyzer и Sentiment Analyzer.

При создании он требует функцию предобработки текста, векторайзер для векторизации текста и классификатор. После обучения в качестве предсказания он будет выдавать вероятность принадлежности сообщения к классу 1.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [16]:
class TextClassifier(object):
    
    def __init__(self, preprocess_text, vectorizer, classifier):
        self.preprocess_text = np.vectorize(preprocess_text)
        self.vectorizer = vectorizer
        self.classifier = classifier
        
    def fit(self, X, y):
        
        print('Text preprocessing...', end=' ')
        X_preprocessed = self.preprocess_text(X)
        print('Done.')
        
        print('Text vectorizing...', end=' ')
        X_vectorized = self.vectorizer.fit_transform(X_preprocessed)
        print('Done.')
        
        print('Fitting classifier...', end=' ')
        self.classifier.fit(X_vectorized, y)
        print('Done.')
        
        return None
    
    def predict(self, X, y=None):
        
        X_preprocessed = self.preprocess_text(X)
        X_vectorized = self.vectorizer.transform(X_preprocessed)        
        y_proba = self.classifier.predict_proba(X_vectorized)
        y_proba = y_proba[:, 1]
        
        return y_proba


Функция для оценки качества вероятностного предсказания на тестовой выборке:

In [17]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    crosstab = pd.crosstab(y_test_true, y_test_pred)
    crosstab.index = pd.Index([0, 1], name='true')
    crosstab.columns = pd.Index([0, 1], name='pred')
    print(crosstab)

**Обучение Message Analyzer**

Разобьем объединенный датафрейм с сообщениями и отзывами на обучающую и тестовую выборки:

In [18]:
X = logs_and_reviews['Message']
y = logs_and_reviews['is_review']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

В качестве векторайзера зададим CountVectorizer, а в качестве классификатора – логистическую регрессию:

In [19]:
message_analyzer = TextClassifier(
    preprocess_text=preprocess_text,
    vectorizer = CountVectorizer(analyzer='word'),
    classifier = LogisticRegression(
        C=10.,
        n_jobs=-1,
        random_state=42,
        )
)

Процесс обучения:

In [20]:
message_analyzer.fit(X_train, y_train)

Preprocessing texts... Done.
Vectorizing texts... Done.
Fitting classifier... Done.


Проверка качества Message Analyzer на тестовой выборке:

In [21]:
y_proba = message_analyzer.predict(X_test)
get_classification_report(y_test, y_proba > 0.5)

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      4951
           1       0.96      0.94      0.95      4937

    accuracy                           0.95      9888
   macro avg       0.95      0.95      0.95      9888
weighted avg       0.95      0.95      0.95      9888

CONFUSION MATRIX

pred     0     1
true            
0     4756   195
1      318  4619


Проверим, как Message Analyzer классифицирует сообщения:

In [22]:
for message in ('Дарт Вейдер', 'Приложение агонь'):
    prediction = message_analyzer.predict([message])
    prediction = prediction[0]

    if prediction >= 0.5:
        print(f'"{message}": Review')
    else:
        print(f'"{message}": Chat')

"Дарт Вейдер": Chat
"Приложение агонь": Review


**Обучение Sentiment Analyzer**

Разобьем датафрейм с отзывами на обучающую и тестовую выборки:

In [23]:
X = reviews['Message']
y = reviews['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

Поскольку это стадия анализа тональности, зададим расширенное множество стоп-слов. Исключим из него слова из базового множества, частицу «не» и др. В дальнейшем для улучшения качества анализа в него таким же образом можно будет добавлять или убирать новые слова.

In [24]:
manual_stop = nltk.corpus.stopwords.words('russian')
manual_stop = set([normalize_word(word) for word in manual_stop])
manual_stop -= basic_stop
manual_stop -= {'не', 'хорошо', 'хороший'}

В качестве векторайзера зададим CountVectorizer (в который передадим расширенное множество стоп-слов) с возможностью обучения также на биграммах и триграммах, в качестве классификатора – снова логистическую регрессию:

In [25]:
sentiment_analyzer = TextClassifier(
    preprocess_text=preprocess_text,
    vectorizer = CountVectorizer(
        analyzer='word',
        ngram_range=(1, 3),
        stop_words=manual_stop,
        ),
    classifier = LogisticRegression(
        class_weight='balanced',
        n_jobs=-1,
        random_state=42,
        )
)

Процесс обучения:

In [26]:
sentiment_analyzer.fit(X_train, y_train)

Preprocessing texts... Done.
Vectorizing texts... Done.
Fitting classifier... Done.


Проверка качества Sentiment Analyzer на тестовой выборке:

In [27]:
y_proba = sentiment_analyzer.predict(X_test)
get_classification_report(y_test, y_proba > 0.5)

              precision    recall  f1-score   support

           0       0.74      0.83      0.78       756
           1       0.97      0.95      0.96      4181

    accuracy                           0.93      4937
   macro avg       0.86      0.89      0.87      4937
weighted avg       0.93      0.93      0.93      4937

CONFUSION MATRIX

pred    0     1
true           
0     628   128
1     218  3963


Проверим, как Sentiment Analyzer классифицирует положительный и отрицательный отзыв:

In [28]:
positive_review = 'Приложение замечательное, оплата коммунальных бумажек просто песня, \
    через QR код все само вносится, единственное что надо руками показания счетчика \
    забить и все! Только жми далее и подтвердить. На все платежки пару минут потратил. \
    Я только из за мобильного приложения перешел в сберыч из ВТБ. Там просто ад, вообще \
    ничего не работает нормально в их мобильном приложении. Короче, браво Сбербанк, надеюсь \
    и дальше ваша программа будет работать как швейцарские часы'

negative_review = 'Обновления 11.2.х превращает телефон в горячий кирпич. \
    Работать невозможно, постоянные тормоза. Ни позвонить, ни смс отправить. \
    В лучших традициях каспера. Сделайте возможность отключения антивируса \
    в настройках, кому он не нужен. Либо добавьте возможность прерывания фоновой \
    проверки на вирусы. Не все с Нот20 ультра ходят, у многих средненькие аппрараты, \
    которые приложение "кладет на лопатки". В версии 11.0 таких тормозов не было, \
    началось с обновления 11.2.'


In [29]:
sentiment_analyzer.predict([positive_review])

array([0.98413778])

In [30]:
sentiment_analyzer.predict([negative_review])

array([3.98289317e-05])

Отзывы классифицированы корректно.

Сохраним объекты с помощью pickle для того, чтобы ими можно было воспользоваться далее:

In [33]:
import pickle

with open('message_analyzer.pickle', 'wb') as f:
    pickle.dump(message_analyzer, f)

with open('sentiment_analyzer.pickle', 'wb') as f:
    pickle.dump(sentiment_analyzer, f)

# 2. Telegram и Dialogflow

In [31]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import dialogflow

Зададим необходимые настройки для работы Telegram и Dialogflow, а также пороги для принятия решений по вероятностным предсказаниям от Message Analyzer и Sentiment Analyzer.
<br>Кроме того, зададим списки ответов на отрицательные, нейтральные и негативные отзывы:

In [32]:
TOKEN = '1372658763:AAEFXhA4bjsnZLXC9ewnuDUW3KvxPEexuyM'
updater = Updater(TOKEN, use_context=False) # Токен API к Telegram
dispatcher = updater.dispatcher

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'vnnlp-bot-kpwr-cace54ccce30.json' # скачанный JSON

DIALOGFLOW_PROJECT_ID = 'vnnlp-bot-kpwr' # PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'VNNLP_bot'  # ID бота из телеграма

MIN_REVIEW_LENGTH = 25
IS_REVIEW_TRHESHOLD = 0.5
POSITIVE_THRESHOLD = 0.75
NEGATIVE_THRESHOLD = 0.25

responces_to_positives = [
    'Благодарим за отзыв! Нам очень приятно получать хорошие слова от наших пользователей.',
]

responces_to_negatives = [
    'Спасибо за отзыв! Свяжитесь с нами, постараемся вместе разобраться с этой проблемой.',
]

responces_to_neutrals = [
    'Спасибо за отзыв! Если есть время, напишите, пожалуйста, подробнее.',
]

Логика работы чат-бота:
- чат-боту в Telegram приходит сообщение;
- Message Analyzer проверяет по своему вероятностному порогу, является ли оно отзывом;
- если сообщение является отзывом, и его длина не меньше установленной:
- - Sentiment Analyzer проверяет по вероятностным порогам, является ли отзыв положительным или отрицательным, в противном случае – нейтральным;
- - в зависимости от этого берется случайный ответ из соответствующего спика;
- если сообщение не является отзывом или отзыв слишком короткий, то по нему работает проект, настроенный в Dialogflow (Small-Talk, а также интент, включающий музыку с youtube).


In [34]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Привет!')

def textMessage(bot, update):
    
    message = update.message.text
    prediction = message_analyzer.predict([message])
    prediction = prediction[0]
    is_review = prediction >= IS_REVIEW_TRHESHOLD

    if is_review and len(message) >= MIN_REVIEW_LENGTH:
        
        prediction = sentiment_analyzer.predict([message])
        prediction = prediction[0]
        
        if prediction >= POSITIVE_THRESHOLD:
            text = np.random.choice(responces_to_positives)
        elif prediction <= NEGATIVE_THRESHOLD:
            text = np.random.choice(responces_to_negatives)
        else:
            text = np.random.choice(responces_to_neutrals)
            
        text = f'{text} \nС уважением, команда бота.'

    else:
        session_client = dialogflow.SessionsClient()
        session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
        text_input = dialogflow.types.TextInput(text=update.message.text,
                                                language_code=DIALOGFLOW_LANGUAGE_CODE)
        query_input = dialogflow.types.QueryInput(text=text_input)
        try:
            response = session_client.detect_intent(session=session, query_input=query_input)
        except InvalidArgument:
             raise

        text = response.query_result.fulfillment_text
        if not text:
            text = 'Что?'
    
    bot.send_message(chat_id=update.message.chat_id,
                     text=text)    


Запуск чат-бота:

In [ ]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()